# 卷积神经网络开放题

# 数据集
本次开放题将与课程内容保持一致，将使用图像数据集Fashion-MNIST [1] 进行计算机视觉任务的设计，该数据集由衣服、鞋子等服饰组成，共10个类别。

这里简介将此数据集转换成卷积神经网络所需要的输入格式的方法：

## 加载数据集

首先导入本作业需要的包或模块。

In [1]:
import torchvision
import torch
from matplotlib import pyplot as plt
from IPython import display
import torch.nn as nn
import time
import torch.nn.functional as F

通过`load_data_fashion_mnist`函数对数据集进行加载，另外函数还指定了参数`transform = transforms.ToTensor()`使所有数据转换为`Tensor`，如果不进行转换则返回的是PIL图片。`transforms.ToTensor()`将尺寸为 (H x W x C) 且数据位于 [0, 255] 的PIL图片或者数据类型为`np.uint8`的NumPy数组转换为尺寸为 (C x H x W) 且数据类型为`torch.float32`且位于 [0.0, 1.0] 的`Tensor`。

我们将在训练数据集上训练模型，并将训练好的模型在测试数据集上评价模型的表现。函数中`mnist_train`是`torch.utils.data.Dataset`的子类，所以我们可以将其传入`torch.utils.data.DataLoader`来创建一个读取小批量数据样本的`DataLoader`实例。

在实践中，数据读取经常是训练的性能瓶颈，特别当模型较简单或者计算硬件性能较高时。PyTorch的`DataLoader`中一个很方便的功能是允许使用多进程来加速数据读取。这里我们通过参数`num_workers`来设置进程读取数据。

In [2]:
def load_data_fashion_mnist(batch_size, resize=None, root="/home/kesci/input/FashionMNIST2065/"):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())

    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

    return train_iter, test_iter

对于AlexNet，我们需要将 Fashion-MNIST 数据集的图像高和宽扩大到224，这个可以通过在`load_data_fashion_mnist`中传入`Resize`来实现，这边设置数据的`batch_size`为128。

In [3]:
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

Fashion-MNIST中一共包括了10个类别，分别为t-shirt（T恤）、trouser（裤子）、pullover（套衫）、dress（连衣裙）、coat（外套）、sandal（凉鞋）、shirt（衬衫）、sneaker（运动鞋）、bag（包）和ankle boot（短靴）。以下函数可以将数值标签转成相应的文本标签。

In [4]:
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

下面定义一个可以在一行里画出多张图像和对应标签的函数。

In [5]:
def show_fashion_mnist(images, labels):
    """Use svg format to display plot in jupyter"""
    display.set_matplotlib_formats('svg')
    # 这里的_表示我们忽略（不使用）的变量
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((224, 224)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)

读取训练数据集中第一个`batch`的数据。

In [6]:
train_data = iter(train_iter)
images, labels = next(train_data)

现在，我们看一下训练数据集中前10个样本的图像内容和文本标签。

In [7]:
labels = get_fashion_mnist_labels(labels)
show_fashion_mnist(images[:10], labels[:10])
plt.show()

<Figure size 864x864 with 10 Axes>

# 卷积神经网络（CNN）

对于计算机视觉的分类任务，在很长一段时间里流行的是研究者通过经验与智慧所设计并生成的手工特征。这类图像分类研究的主要流程是：

1. 获取图像数据集；
2. 使用已有的特征提取函数生成图像的特征；
3. 使用机器学习模型对图像的特征分类。

卷积神经网络就是含卷积层的神经网络，深度卷积神经网络的兴起改变了计算机视觉任务中手工设计的特征的传统，引领了诸多影响深远的研究。

## LeNet



LeNet [2] 作为一个早期用来识别手写数字图像的卷积神经网络，展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。如下图所示：


![LeNet模型](https://d2l.ai/_images/lenet.svg)


LeNet的模型结构分为卷积层块和全连接层块两个部分：

- 卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别，并且通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。

- 全连接层块将卷积层块的输出中每个样本变平（flatten），即输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，从而进行分类。



## AlexNet

2012年，AlexNet [3] 横空出世，使用了8层卷积神经网络，并以很大的优势赢得了ImageNet 2012图像识别挑战赛。

AlexNet与LeNet的设计理念非常相似，但相对较小的LeNet相比，AlexNet包含5层卷积和2层全连接隐藏层，以及1个全连接输出层，模型参数也大大增加。由于早期显存的限制，最早的AlexNet使用双数据流的设计，使一个GPU只需要处理一半模型。如下图所示：

![AlexNet模型](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter05/5.6_alexnet.png)

AlexNet首次证明了神经网络以端到端（end-to-end）的方式学习到的特征可以超越手工设计的特征，从而一举打破计算机视觉研究的前状。



### 问题一：
- 阅读提出上述两种网络的相关论文，试从数据集的预处理、激活函数的使用、训练方法的改进以及模型结构的变化等角度，从理论层面分析比较LeNet与AlexNet的结构差异，并尝试解释AlexNet为什么会具有对计算机视觉任务优越的处理性能。
- AlexNet对Fashion-MNIST数据集来说可能过于复杂，请尝试对模型进行简化来使训练更快，同时保证分类准确率（accuracy）不明显下降（不低于85%），并将简化后的结构、节省的训练时间以及下降的准确率等相关指标以表格的形式进行总结分析。

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
         # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),#两个参数：输入神经元个数，输出神经元个数；256：输入通道数，5*5输入map的宽和高
            nn.ReLU(),
            nn.Dropout(0.5),
            #由于使用CPU镜像，精简网络，若为GPU镜像可添加该层
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )

    def forward(self, img):#把上面两个模块self.conv和self.fc连接起来
#self.conv的输出是一个四维张量：batchsize*out_channels*height*width
#self.fc的输入需要的是一个二维张量：batchsize*hiddens
#所以本函数需要做一个reshape的工作
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))#img.shape[0]=batchsize
        return output

In [9]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n
    
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

In [35]:
net=AlexNet()
lr, num_epochs = 0.001, 3#相比LeNet,我们把AlexNet的学习率调小了

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
#d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.6038, train acc 0.771, test acc 0.853, time 131.1 sec
epoch 2, loss 0.3336, train acc 0.876, test acc 0.876, time 131.0 sec
epoch 3, loss 0.2880, train acc 0.893, test acc 0.886, time 131.3 sec


In [25]:
#简化的AlexNet网络
class AlexNet1(nn.Module):
    def __init__(self):
        super(AlexNet1, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
           # nn.Conv2d(384, 384, 3, 1, 1),
            #nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
         # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),#两个参数：输入神经元个数，输出神经元个数；256：输入通道数，5*5输入map的宽和高
            nn.ReLU(),
            nn.Dropout(0.5),
            #由于使用CPU镜像，精简网络，若为GPU镜像可添加该层
            #nn.Linear(4096, 4096),
            #nn.ReLU(),
            #nn.Dropout(0.5),

            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )

    def forward(self, img):#把上面两个模块self.conv和self.fc连接起来
#self.conv的输出是一个四维张量：batchsize*out_channels*height*width
#self.fc的输入需要的是一个二维张量：batchsize*hiddens
#所以本函数需要做一个reshape的工作
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))#img.shape[0]=batchsize
        return output

In [26]:
net=AlexNet1()
lr, num_epochs = 0.001, 3#相比LeNet,我们把AlexNet的学习率调小了

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.5091, train acc 0.811, test acc 0.871, time 110.5 sec
epoch 2, loss 0.3005, train acc 0.886, test acc 0.891, time 109.7 sec
epoch 3, loss 0.2558, train acc 0.904, test acc 0.894, time 110.5 sec


## VGG

AlexNet在LeNet的基础上增加了3个卷积层，同时对网络的卷积窗口、输出通道数和构造顺序均做了大量的调整。AlexNet指明了深度卷积神经网络可以取得出色的结果，基于这一个理念，牛津大学的实验室Visual Geometry Group实验室提出了VGG [4] 网络，提供了通过重复使用简单的基础块来构建深度模型的思路。VGG每个基础块组成规律是：连续使用数个相同的填充为1、窗口形状为$3\times 3$的卷积层后接上一个步幅为2、窗口形状为$2\times 2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。如下图所示：

![VGG模型](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/vgg.jpg)

可以看到，每次经过基础块以后，网络会将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

### 问题二：
- LeNet与AlexNet在接近图像输入的卷积模块中都引入了较大尺寸的卷积核，如$5\times 5$或者$7\times 7$的卷积窗口来捕捉更大范围的图像信息，试分析VGG每个基础块的固定设计是否会影响到图像的粗粒度信息提取，并且对比不同结构模块输出的特征图进行对比。
- 尝试将Fashion-MNIST中图像的高和宽由224改为96，试分析VGG网络的参数变化情况，并且对比模型训练时间、分类准确率（accuracy）等实验指标受到的影响，以表格的形式进行总结分析。

In [10]:
def vgg_block(num_convs, in_channels, out_channels): #卷积层个数，输入通道数，输出通道数
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(kernel_size=2, stride=2)) # 这里会使宽高减半
    return nn.Sequential(*blk)

In [11]:
#conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
#conv_arch = ((1, 1, 32), (1, 32, 64), (2, 64, 128), (2, 128, 256), (2, 256, 256))
conv_arch = ((1, 1, 4), (1, 4, 8), (2, 8, 16), (2, 16, 32), (2, 32, 32))
# 经过5个vgg_block, 宽高会减半5次, 变成 224/32 = 7
#fc_features = 512 * 7 * 7 # c * w * h
fc_features = 32 * 7 * 7 # c * w * h
#fc_hidden_units = 4096 # 任意
fc_hidden_units = 256

In [10]:
class FlattenLayer(torch.nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)

In [13]:
def vgg(conv_arch, fc_features, fc_hidden_units=4096):#conv_arch：每个VGGblock的参数设置
    net = nn.Sequential()
    # 卷积层部分
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        # 每经过一个vgg_block都会使宽高减半
        net.add_module("vgg_block_" + str(i+1), vgg_block(num_convs, in_channels, out_channels))#.add_module：拼接
    # 全连接层部分
    net.add_module("fc", nn.Sequential(
                                 #d2l.FlattenLayer(),#展平
                                 FlattenLayer(),#展平
                                 nn.Linear(fc_features, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, 10)
                                ))
    return net

In [23]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
batch_size=16
#batch_size = 64
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

lr, num_epochs = 0.00001, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
#d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.9261, train acc 0.260, test acc 0.642, time 74.8 sec
epoch 2, loss 1.0502, train acc 0.598, test acc 0.698, time 75.1 sec
epoch 3, loss 0.8939, train acc 0.665, test acc 0.720, time 75.0 sec
epoch 4, loss 0.8243, train acc 0.690, test acc 0.734, time 75.1 sec
epoch 5, loss 0.7792, train acc 0.708, test acc 0.740, time 75.1 sec
epoch 6, loss 0.7455, train acc 0.720, test acc 0.748, time 74.9 sec
epoch 7, loss 0.7188, train acc 0.731, test acc 0.755, time 74.9 sec
epoch 8, loss 0.6998, train acc 0.739, test acc 0.757, time 75.1 sec
epoch 9, loss 0.6787, train acc 0.744, test acc 0.765, time 74.9 sec
epoch 10, loss 0.6578, train acc 0.753, test acc 0.764, time 75.0 sec


In [17]:
#改变fashionminst数据集的图片大小为96
batch_size=16
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=96)

In [18]:
conv_arch = ((1, 1, 4), (1, 4, 8), (2, 8, 16), (2, 16, 32), (2, 32, 32))
# 经过5个vgg_block, 宽高会减半5次, 变成 96/32 = 3
fc_features = 32* 3* 3 # c * w * h
fc_hidden_units = 256 # 任意

In [20]:
net = vgg(conv_arch, fc_features, fc_hidden_units)

lr, num_epochs = 0.0001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.9703, train acc 0.624, test acc 0.772, time 75.0 sec
epoch 2, loss 0.5821, train acc 0.781, test acc 0.813, time 74.7 sec
epoch 3, loss 0.4940, train acc 0.817, test acc 0.836, time 74.8 sec
epoch 4, loss 0.4440, train acc 0.835, test acc 0.852, time 75.1 sec
epoch 5, loss 0.4105, train acc 0.848, test acc 0.856, time 75.1 sec


## NiN

之前介绍的LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。网络中的网络（NiN）[5] 提出了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。

卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维，$1\times 1$卷积层可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用$1\times 1$卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。下图对比了NiN同AlexNet和VGG等网络在结构上的主要区别。

![左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部](http://zh.d2l.ai/_images/nin.svg)

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的$1\times 1$卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。


## GoogLeNet

在2014年的ImageNet图像识别挑战赛中，一个名叫GoogLeNet的网络结构大放异彩 [6] ，它虽然在名字上向LeNet致敬，但在网络结构上已经很难看到LeNet的影子。GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上NiN块相比，这个基础块在结构上更加复杂，如下图所示：

![Inception块的结构](http://zh.d2l.ai/_images/inception.svg)

Inception块里有4条并行的线路。前3条线路使用窗口大小分别是$1\times 1$、$3\times 3$和$5\times 5$的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做$1\times 1$卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用$3\times 3$最大池化层，后接$1\times 1$卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后将每条线路的输出在通道维上连结，并输入接下来的层中去。Inception块中可以自定义的超参数是每个层的输出通道数，以此来控制模型复杂度。

### 问题三：
- 对比AlexNet、VGG和NiN、GoogLeNet的模型参数尺寸，从理论的层面分析为什么后两个网络可以显著减小模型参数尺寸？
- GoogLeNet有数个后续版本，包括加入批量归一化层 [7]、对Inception块做调整 [8] 和加入残差连接 [9]，请尝试实现并运行它们，然后观察实验结果，以表格的形式进行总结分析。

# GoogLeNet
1. 由Inception基础块组成。  
2. Inception块相当于⼀个有4条线路的⼦⽹络。它通过不同窗口形状的卷积层和最⼤池化层来并⾏抽取信息，并使⽤1×1卷积层减少通道数从而降低模型复杂度。   
3. 可以⾃定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。 
1×1卷积层的作用是减少通道数，降低模型复杂度（和NiN的1×1卷积层的作用不同，NiN的作用的当作全连接层使用）

![Image Name](https://cdn.kesci.com/upload/image/q5l6uortw.png?imageView2/0/w/640/h/640)


In [22]:
class Inception(nn.Module):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)#用padding的方式保持形状不变
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)  # 在通道维上连结输出

###GoogLeNet模型
完整模型结构  
输入的图形大小为：1×96×96

![Image Name](https://cdn.kesci.com/upload/image/q5l6x0fyyn.png?imageView2/0/w/640/h/640)


In [11]:
class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

In [24]:
#b1：第一和第二个方框：抽取特征减少大小
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
#b2：第3——第5个方框
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),#没有增加通道数，起到非线性的作用
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),#增加通道数，保持形状不改变
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),#196为输入通道数，64+128+32+32=256为c输出通道数，是下一个inception的输入通道数
                   Inception(256, 128, (128, 192), (32, 96), 64),#256为输入通道数，128+192+96+64=480输出通道数
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),#128+192+96+64=480输入通道数下面同理
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   GlobalAvgPool2d())

net = nn.Sequential(b1, b2, b3, b4, b5, 
                    FlattenLayer(), nn.Linear(1024, 10))

net = nn.Sequential(b1, b2, b3, b4, b5, FlattenLayer(), nn.Linear(1024, 10))

X = torch.rand(1, 1, 96, 96)

for blk in net.children(): 
    X = blk(X)
    print('output shape: ', X.shape)


#batchsize=128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 12, 12])
output shape:  torch.Size([1, 480, 6, 6])
output shape:  torch.Size([1, 832, 3, 3])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])
training on  cuda
epoch 1, loss 1.0111, train acc 0.607, test acc 0.793, time 615.1 sec
epoch 2, loss 0.3862, train acc 0.858, test acc 0.857, time 614.6 sec
epoch 3, loss 0.3285, train acc 0.876, test acc 0.884, time 614.2 sec
epoch 4, loss 0.2912, train acc 0.892, test acc 0.891, time 615.8 sec
epoch 5, loss 0.2644, train acc 0.901, test acc 0.905, time 613.4 sec


加入批量归一化层

In [25]:
def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    #X：我们的标准化目标，是（全连接层仿射变换/卷积计算）的输出，我们希望把它标准化为期望是0，方差是1的X_hat
    #momentum：超参数不需要学习，是一个动量
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

In [26]:
class BatchNorm(nn.Module):#用来维护一些学习参数和超参数
    def __init__(self, num_features, num_dims):
        #num_features：全连接层代表输出神经元个数，卷积层代表通道数
        #num_dims：全连接层等于2，卷积层等于4
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features) #全连接层输出神经元
        else:
            shape = (1, num_features, 1, 1)  #通道数
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training, 
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

In [32]:
#b1：第一和第二个方框：抽取特征减少大小
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   BatchNorm(64, num_dims=4),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
#b2：第3——第5个方框
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),#没有增加通道数，起到非线性的作用
                   BatchNorm(64, num_dims=4),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),#增加通道数，保持形状不改变
                   BatchNorm(192, num_dims=4),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),#196为输入通道数，64+128+32+32=256为c输出通道数，是下一个inception的输入通道数
                   BatchNorm(256, num_dims=4),
                   Inception(256, 128, (128, 192), (32, 96), 64),#256为输入通道数，128+192+96+64=480输出通道数
                   BatchNorm(480, num_dims=4),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),#128+192+96+64=480输入通道数下面同理
                   BatchNorm(512, num_dims=4),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   BatchNorm(512, num_dims=4),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   BatchNorm(512, num_dims=4),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   BatchNorm(528, num_dims=4),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   BatchNorm(832, num_dims=4),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   BatchNorm(832, num_dims=4),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   BatchNorm(1024, num_dims=4),
                   GlobalAvgPool2d())

net = nn.Sequential(b1, b2, b3, b4, b5,FlattenLayer(), nn.Linear(1024, 10))

X = torch.rand(1, 1, 96, 96)

for blk in net.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

#batchsize=128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 12, 12])
output shape:  torch.Size([1, 480, 6, 6])
output shape:  torch.Size([1, 832, 3, 3])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])
training on  cuda
epoch 1, loss 0.4528, train acc 0.834, test acc 0.874, time 962.7 sec
epoch 2, loss 0.3070, train acc 0.889, test acc 0.903, time 960.8 sec
epoch 3, loss 0.2626, train acc 0.904, test acc 0.890, time 961.4 sec
epoch 4, loss 0.2305, train acc 0.916, test acc 0.900, time 961.1 sec
epoch 5, loss 0.2153, train acc 0.921, test acc 0.909, time 962.4 sec


基于[8]对Inception块做调整
论文提出四个原则：
原则１:设计网络的时候需要避免 representational bottlenecks;　什么意思呢?　文章中说： 层与层之间进行 information 传递时，要避免这个过程中的数据的extreme compression，也就是说，数据的 scale 不能减小的太快；（数据从输入到输出大致是减少的，这个变化过程一定要gently，而不是快速的，    一定是慢慢的变少。。。。。。）       当数据的维数extreme下降的时候，就相当于引入了 representational bottelneck.
原则2：没有怎么看明白什么意思啊？复制过来。Higher dimensional representations are easier to process locally within a network. Increasing the activations per tile in a convolutional network allows for more 
disentangled features. The resulting networks will train faster.  （可以结合 figure7 下面的注释， 我感觉： 在高维表示时，对于局部的特征更容易处理，意思就是local 卷积，用1*1啦， 或3*3, 别用太大的）
原则3： spatial aggregation can be done over lower dimensional embedding without much or any loss in representational power.    直接翻译真的不会翻译啊
原则4： 应该均衡网络的宽度与深度；


网络改进：
1. 把大的卷积层分解为小的卷积层，提高计算效率：
第一种：可以把一个5*5的卷积卷积层分解成两个 3*3 的卷积层。       
一个细节就是：把底层的 filters 为m 时， 上层的filters 为 n 时，这时两层的小的卷积层的每一个filters 为多少呢？ 细节2： 当原来的 激活函数为线性激活函数时，现在变为两层的激活函数如何选择？（文中说明了全部使用 relu 激活函数会好一些）

![Image Name](http://images2015.cnblogs.com/blog/961754/201706/961754-20170609114156012-76460819.png)

In [12]:
class Inception1(nn.Module):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception1, self).__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)#用padding的方式保持形状不变
        # 线路3，1 x 1卷积层后接两个3 x 3卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=3, padding=1)
        self.p3_3 = nn.Conv2d(c3[1], c3[1], kernel_size=3, padding=1)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_3(F.relu(self.p3_2(F.relu(self.p3_1(x))))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)  # 在通道维上连结输出

In [15]:
#b1：第一和第二个方框：抽取特征减少大小
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
#b2：第3——第5个方框
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),#没有增加通道数，起到非线性的作用
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),#增加通道数，保持形状不改变
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception1(192, 64, (96, 128), (16, 32), 32),#196为输入通道数，64+128+32+32=256为c输出通道数，是下一个inception的输入通道数
                   Inception1(256, 128, (128, 192), (32, 96), 64),#256为输入通道数，128+192+96+64=480输出通道数
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception1(480, 192, (96, 208), (16, 48), 64),#128+192+96+64=480输入通道数下面同理
                   Inception1(512, 160, (112, 224), (24, 64), 64),
                   Inception1(512, 128, (128, 256), (24, 64), 64),
                   Inception1(512, 112, (144, 288), (32, 64), 64),
                   Inception1(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception1(832, 256, (160, 320), (32, 128), 128),
                   Inception1(832, 384, (192, 384), (48, 128), 128),
                   GlobalAvgPool2d())

net1 = nn.Sequential(b1, b2, b3, b4, b5, FlattenLayer(), nn.Linear(1024, 10))

X = torch.rand(1, 1, 96, 96)

for blk in net1.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

#batchsize=128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net1.parameters(), lr=lr)
train_ch5(net1, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 12, 12])
output shape:  torch.Size([1, 480, 6, 6])
output shape:  torch.Size([1, 832, 3, 3])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])
training on  cuda
epoch 1, loss 0.9698, train acc 0.626, test acc 0.781, time 669.8 sec
epoch 2, loss 0.4199, train acc 0.842, test acc 0.855, time 672.0 sec
epoch 3, loss 0.3421, train acc 0.870, test acc 0.851, time 668.5 sec
epoch 4, loss 0.3067, train acc 0.885, test acc 0.879, time 668.6 sec
epoch 5, loss 0.2851, train acc 0.893, test acc 0.885, time 668.9 sec


 2.非对称分解：
把一个 n*n 的卷积层分解为两个 1*N 和 N*1 的卷积层；         （文中说了这种分解在网络的开始几层效果垃圾， but is gives very good result on medium grid-sizes）
![Image Name](http://images2015.cnblogs.com/blog/961754/201706/961754-20170609114204653-1728991877.png)


In [ ]:
class Inception2(nn.Module):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=(1,3), padding=1)#用padding的方式保持形状不变
        self.p2_3 = nn.Conv2d(c2[1], c2[1], kernel_size=(3,1))
        # 线路3，1 x 1卷积层后接两个3 x 3卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=(1,3), padding=1)
        self.p3_3 = nn.Conv2d(c3[1], c3[1], kernel_size=(3,1))
        self.p3_4 = nn.Conv2d(c3[1], c3[1], kernel_size=(1,3), padding=1)
        self.p3_5 = nn.Conv2d(c3[1], c3[1], kernel_size=(3,1))
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_3(F.relu(self.p2_2(F.relu(self.p2_1(x))))))
        p3 = F.relu(self.p3_5(F.relu(self.p3_4(F.relu(self.p3_3(F.relu(self.p3_2(F.relu(self.p3_1(x))))))))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)  # 在通道维上连结输出

In [ ]:
#b1：第一和第二个方框：抽取特征减少大小
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
#b2：第3——第5个方框
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),#没有增加通道数，起到非线性的作用
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),#增加通道数，保持形状不改变
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception2(192, 64, (96, 128), (16, 32), 32),#196为输入通道数，64+128+32+32=256为c输出通道数，是下一个inception的输入通道数
                   Inception2(256, 128, (128, 192), (32, 96), 64),#256为输入通道数，128+192+96+64=480输出通道数
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception2(480, 192, (96, 208), (16, 48), 64),#128+192+96+64=480输入通道数下面同理
                   Inception2(512, 160, (112, 224), (24, 64), 64),
                   Inception2(512, 128, (128, 256), (24, 64), 64),
                   Inception2(512, 112, (144, 288), (32, 64), 64),
                   Inception2(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception2(832, 256, (160, 320), (32, 128), 128),
                   Inception2(832, 384, (192, 384), (48, 128), 128),
                   GlobalAvgPool2d())

net2 = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))

X = torch.rand(1, 1, 96, 96)

for blk in net2.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

#batchsize=128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net2.parameters(), lr=lr)
train_ch5(net2, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

加入残差块[9]

In [13]:
class Residual(nn.Module):  # 本类已保存在d2lzh_pytorch包中方便以后使用
    #可以设定输出通道数、是否使用额外的1x1卷积层来修改通道数以及卷积层的步幅。
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)#用padding的方式保持形状不变
        # 线路3，1 x 1卷积层后接两个3 x 3卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=3, padding=1)
        self.p3_3 = nn.Conv2d(c3[1], c3[1], kernel_size=3, padding=1)
        
        #self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        #self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
            
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.bn3 = nn.BatchNorm2d(out_channels)
        
    def forward(self, X):
        if self.conv3:
            X = self.conv3(X)
        p1 = F.relu(self.bn1(self.p1_1(x))+X)
        p2 = F.relu(self.bn2(self.p2_2(F.relu(self.p2_1(x))))+X)
        p3 = F.relu(self.bn3(self.p3_3(F.relu(self.p3_2(F.relu(self.p3_1(x))))))+X)
        #return torch.cat((p1, p2, p3), dim=1)  
        #Y = F.relu(self.bn1(self.conv1(X)))
        #Y = self.bn2(self.conv2(Y))
        return torch.cat((p1, p2, p3), dim=1) 

In [14]:
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    #num_residuals残差快个数
    if first_block:
        assert in_channels == out_channels # 第一个模块的通道数同输入通道数一致
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

In [15]:
#b1：第一和第二个方框：抽取特征减少大小
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
#b2：第3——第5个方框
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),#没有增加通道数，起到非线性的作用
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),#增加通道数，保持形状不改变
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception1(192, 64, (96, 128), (16, 32), 32),#196为输入通道数，64+128+32+32=256为c输出通道数，是下一个inception的输入通道数
                   Inception1(256, 128, (128, 192), (32, 96), 64),#256为输入通道数，128+192+96+64=480输出通道数
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception1(480, 192, (96, 208), (16, 48), 64),#128+192+96+64=480输入通道数下面同理
                   Inception1(512, 160, (112, 224), (24, 64), 64),
                   Inception1(512, 128, (128, 256), (24, 64), 64),
                   Inception1(512, 112, (144, 288), (32, 64), 64),
                   Inception1(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception1(832, 256, (160, 320), (32, 128), 128),
                   Inception1(832, 384, (192, 384), (48, 128), 128),
                   GlobalAvgPool2d())
                   
net3 = nn.Sequential(b1, b2, b3, b4, b5)
net3.add_module("resnet_block1", resnet_block(1024, 1024, 2, first_block=True))
net3.add_module("resnet_block2", resnet_block(1024, 512, 2))
net3.add_module("resnet_block3", resnet_block(512, 256, 2))
net3.add_module("resnet_block4", resnet_block(256, 256, 2))
net3.add_module("global_avg_pool", GlobalAvgPool2d()) # GlobalAvgPool2d的输出: (Batch, 512, 1, 1)
net3.add_module("fc", nn.Sequential(FlattenLayer(), nn.Linear(256, 10))) 

In [ ]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net3.parameters(), lr=lr)
train_ch5(net3, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.9511, train acc 0.633, test acc 0.636, time 5632.3 sec


## 参考文献
[1] Xiao, H., Rasul, K., & Vollgraf, R. (2017). Fashion-mnist: a novel image dataset for benchmarking machine learning algorithms. arXiv preprint arXiv:1708.07747.

[2] LeCun, Y., Bottou, L., Bengio, Y., & Haffner, P. (1998). Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11), 2278-2324.

[3] Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012). Imagenet classification with deep convolutional neural networks. In Advances in neural information processing systems (pp. 1097-1105).

[4] Simonyan, K., & Zisserman, A. (2014). Very deep convolutional networks for large-scale image recognition. arXiv preprint arXiv:1409.1556.

[5] Lin, M., Chen, Q., & Yan, S. (2013). Network in network. arXiv preprint arXiv:1312.4400.

[6] Szegedy, C., Liu, W., Jia, Y., Sermanet, P., Reed, S., & Anguelov, D. & Rabinovich, A.(2015). Going deeper with convolutions. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1-9).

[7] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.

[8] Szegedy, C., Vanhoucke, V., Ioffe, S., Shlens, J., & Wojna, Z. (2016). Rethinking the inception architecture for computer vision. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (pp. 2818-2826).

[9] Szegedy, C., Ioffe, S., Vanhoucke, V., & Alemi, A. A. (2017, February). Inception-v4, inception-resnet and the impact of residual connections on learning. In Proceedings of the AAAI Conference on Artificial Intelligence (Vol. 4, p. 12).

## 项目报告
本次大作业的终审评估以项目报告作为重要依据，开放题报告的内容和排版要求请下载文件：


[termproject2.zip](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/termproject2.zip)

需要注意的是，文件中：
- `termproject.pdf`提供了项目报告的内容格式要求
- `termproject_exp.pdf`提供了项目报告的内容排版样例


推荐使用`LaTeX`软件进行报告的撰写，相关`.tex`以及`.sty`源文件一并附于文件夹中。
